Notes
    
        
- Heaviside step function
- def step_fun(x,a,b,c): return a * (np.sign(x-b) + c)
- p0=[np.max(y), np.median(x), np.min(y)]


In [41]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
from scipy.optimize import curve_fit
# from mplcursors import cursor
from dataclasses import dataclass
import copy
print("done")

done


In [42]:
def step_sim_v1(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 300, 
                                lambda x: 100, 
                                lambda x: 50, 
                                lambda x: 350,
                                lambda x: 20,
                                lambda x: 350,
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps

def step_sim_v2(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 600, 
                                lambda x: 600 - ((x+1)%100), 
                                lambda x: 500 - ((x+1)%200), 
                                lambda x: 400 - ((x+1)%300),
                                lambda x: 300 - ((x+1)%400),
                                lambda x: 200 - ((x+1)%500),
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps

def step_sim_v3(step_size = 8.0, SNR = 0.5, total_time_point=400):
    np.random.seed(20181118)
    
    steps = np.piecewise(np.arange(0, total_time_point), 
                            [
                                np.arange(0, total_time_point) < 99, 

                                np.logical_and(np.arange(0, total_time_point) >= 99, np.arange(0, total_time_point) < 199), 

                                np.logical_and(np.arange(0, total_time_point) >= 199, np.arange(0, total_time_point) < 249), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 249, np.arange(0, total_time_point) < 349), 
                                
                                np.logical_and(np.arange(0, total_time_point) >= 349, np.arange(0, total_time_point) < 499), 

                                np.arange(0, total_time_point) >= 499
                            ], 
                            [
                                lambda x: 600, 
                                lambda x: 500, 
                                lambda x: 400 , 
                                lambda x: 300 ,
                                lambda x: 200 ,
                                lambda x: 100 ,
                            ])
    x = np.arange(0, total_time_point)
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(x))
    # y = true_fct + np.random.standard_t(1, size=len(x))
    y = steps + noise
    return x, y, steps
print("done")

done


In [129]:
# Unit Functions
@dataclass(repr=True)
class FitInfo:
    fit_params: list
    y_fit:np.ndarray
    x:np.ndarray = None
    y:np.ndarray = None

@dataclass(repr=True)
class Locations:
    x_indices:list
    y_indices:list
        
@dataclass(repr=True)
class Height:
    height:float
    location:int

        
def moving_average(a, n=2, pad=False) :
    if pad:
        a = np.pad(a, (n-1,0))
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def _step_fun_logistic_v2(x, amplitude=1.0, center=0.0, sigma=1.0):
    tiny = 1.0e-15
    # ymin, ymax = min(data), max(data)
    # xmin, xmax = min(x), max(x)
    # pars = self.make_params(amplitude=(ymax-ymin),
    #                          center=(xmax+xmin)/2.0)
    # pars[f'{self.prefix}sigma'].set(value=(xmax-xmin)/7.0, min=0.0)
    out = (x - center)/max(tiny, sigma)
    out = 1. - 1./(1. + np.exp(out))
    return amplitude*out
    
    
def _step_fun_logistic(x, L ,x0, k, b):
    # Logistic function
    # f(X, L, x0, k, b)
    # https://en.wikipedia.org/wiki/Logistic_function
    # use for curve fitting
    # https://stackoverflow.com/questions/55725139/fit-sigmoid-function-s-shape-curve-to-data-using-python
    # #p0=[np.max(y), np.median(x), 1, np.min(y)]
    y = L / (1 + np.exp(-k*(x-x0))) + b
        #amplitude*(1. - 1./(1. + exp((x-center)/max(tiny, sigma))))
    return y

def _step_fun_heaviside(x, a, b, c): 
    #p0=[np.max(y), np.median(x), np.min(y)]
    return a * (np.sign(x-b) + c)

def fit(x, y, fun, p0, get_data=False):
    popt, pcov, infodict, mesg, ier = curve_fit(fun, x, y,p0=p0, full_output=True, maxfev = 1000000)
    y_fit = fun(x, *popt)
    info = FitInfo(fit_params=popt, y_fit=y_fit)
    if get_data:
        info.x = x
        info.y = y
        
    return info

def get_step_locations(x, y):
    uniques, y_indices, counts = np.unique(y, return_counts=True, return_index=True)
    x_indices = [ x[i] for i in y_indices ]
    return Locations(x_indices=x_indices, y_indices=list(y_indices))

def get_window_heights(x, y, locations:Locations):
    heights = []
    y_locations = copy.deepcopy(locations.y_indices)
    y_locations.sort()
    
    previous_loc = y_locations.pop(0)
    if previous_loc == 0:
        previous_loc = y_locations.pop(0)
        
    previous_mean = np.mean(y[0:previous_loc+1])
    
    while len(y_locations):
        next_loc = y_locations.pop(0)
        current_mean = np.mean(y[previous_loc+1: next_loc+1])
        
        height = abs(previous_mean - current_mean)
        heights.append(Height(height=height, location=previous_loc))
        
        previous_loc = next_loc
        previous_mean = current_mean
        
    if previous_loc != (len(y)-1):
        current_mean = np.mean(y[previous_loc+1: ])
        height = abs(previous_mean - current_mean)
        heights.append(Height(height=height, location=previous_loc))
        
    return heights
        
    
    
    
    

print("done")

done


In [130]:
# Exe Functions


In [131]:
# x_data, y_data , steps_org = step_sim_v2(total_time_point=1000, max_signal=2000)
x_data, y_data , steps_org = step_sim_v3(total_time_point=600)


In [132]:
# ymin, ymax = min(y_data), max(y_data)
# xmin, xmax = min(x_data), max(x_data)
# amplitude=(ymax-ymin)
# center=(xmax+xmin)/2.0
# sigma =(xmax-xmin)/7.0
# p0 = [amplitude, center, sigma]
p0=[np.max(y_data), np.median(x_data), 1, np.min(y_data)]
info = fit(x_data, y_data, _step_fun_logistic, p0=p0)

C:\Users\Zeroth\AppData\Local\Temp\ipykernel_25220\3619110770.py:46: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [133]:
plt.plot(x_data, y_data)
plt.plot(x_data, info.y_fit)


In [134]:
# np.unique(info.y_fit,return_counts=True, return_index=True)

In [135]:
location_r1 = get_step_locations(x_data, info.y_fit)
heights = get_window_heights(x_data, y_data, location_r1)
hh = np.zeros(y_data.shape)
# print(heights)
# print(location_r1)

In [136]:
for h in heights:
    hh[h.location] = h.height
plt.plot(np.arange(len(hh)), hh)

In [137]:
info.fit_params

array([-6.39243508e+02,  2.32333549e+02,  8.04654018e-03,  7.11327732e+02])

In [39]:
plt.plot(np.arange(len(_h)), _h)

In [40]:
np.max(_h)

137.05045641773296

In [78]:
plt.grid()

In [80]:
len(x_data)

600

In [81]:
len(_h)

598

In [90]:
part_1 = y_data[0:100]
part_x = np.arange(len(part_1))
p0=[np.max(part_1), np.median(part_x), 1, np.min(part_1)]
info_1 = fit(part_x, part_1, _step_fun_logistic, p0=p0)
location_r1 = get_step_locations(part_x, info_1.y_fit)
heights = get_window_heights(part_x, part_1, location_r1)
_h = [h.height for h in heights]

C:\Users\Zeroth\AppData\Local\Temp\ipykernel_25220\3619110770.py:46: RuntimeWarning: overflow encountered in exp
  y = L / (1 + np.exp(-k*(x-x0))) + b


In [91]:
plt.plot(part_x, part_1, label="s")
plt.plot(part_x, info_1.y_fit, label="f")
plt.plot(np.arange(len(_h)), _h, label="h")
plt.legend()
plt.grid()

In [88]:
len(info_1.y_fit)

600

In [94]:
np.std(part_1)**2

294.27540308142073

In [93]:
np.max(_h)

16.248045367450118

In [6]:
from typing import Any
class Parameters:

    def __getattr__(self, name):
        raise AttributeError("%r object has no attribute %r" % (self.__class__.__name__, name))
    
    def __setattr__(self, __name: str, __value: Any) -> None:
        self.__dict__[__name] = __value


In [7]:
a = Parameters()
a.init = 10
a.something = 20
print(a.something)

20


In [8]:
a.init

10